In [9]:
import pandas as pd
import numpy as np
from time import time
import json
from sqlalchemy import create_engine
import re
import datetime
#import lifetimes
#import pymc3 as pm

%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns; sns.set()

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

In [2]:
# Set up the SQL connection
dbname = 'redshift'
url = 'panamera-ods.cnsuxis6zqxr.us-west-2.redshift.amazonaws.com:5439/panameraods'
engine = create_engine("postgresql://" + 'lucasdresl' + ":" + 'cQsY801a76x82x62aWDT' + "@" + url)

# Transition matrix with replies

## L2 matrix

Query description : Get for each user the amount of replies in each category

In [35]:
query = """

drop table if exists repliers;
select
  panamera_user_sk
   into temp table repliers
from ods.fact_replies
where country_sk='olx|asia|in'
 and reply_num_panamera=1
and date_sent_nk between '2019-07-01' and '2019-10-01'
order by random() limit 50000

;drop table if exists users_replies;
select
  a.panamera_user_sk,
  category_l2_name_en,
  count(distinct reply_nk) as replies
    into temp table users_replies
   from ods.fact_replies a
    left join ods.dim_categories b on a.category_sk=b.category_sk and b.country_sk='olx|asia|in'
    join repliers c on a.panamera_user_sk=c.panamera_user_sk
 where
       a.country_sk='olx|asia|in'
   and reply_num_panamera=1
   and date_sent_nk between '2019-07-01' and '2019-10-01'
group by 1,2

;drop table if exists users_replies2;
select
  a.panamera_user_sk ,
  category_l2_name_en category_l2_name_en2,
  count(distinct reply_nk) as replies2
    into temp table users_replies2
   from ods.fact_replies a
    left join ods.dim_categories b on a.category_sk=b.category_sk and b.country_sk='olx|asia|in'
    join repliers c on a.panamera_user_sk=c.panamera_user_sk
 where
       a.country_sk='olx|asia|in'
   and reply_num_panamera=1
   and date_sent_nk between '2019-07-01' and '2019-10-01'
group by 1,2

;drop table if exists df;
select
  a.panamera_user_sk,
  a.category_l2_name_en,
  b.category_l2_name_en2,
  b.replies2
 into temp table df
    from users_replies a
 join users_replies2 b on a.panamera_user_sk=b.panamera_user_sk
order by a.panamera_user_sk,a.category_l2_name_en;

select * from df



"""

In [36]:
df = pd.read_sql_query(query.replace('%', '%%'), engine)

In [124]:
df1 = df.pivot_table(index='category_l2_name_en' , columns='category_l2_name_en2' , values='replies2' , aggfunc='sum').fillna(0)
df1 = df1.apply(lambda x:(x/sum(x)) * 100 , axis=1)

In [125]:
df1.head()

category_l2_name_en2,ACs,Accessories,Accountant,BPO & Telecaller,Beds & Wardrobes,Bicycles,Books,Cameras & Lenses,Cars,Commercial Vehicles,Computer Accessories,Computers & Laptops,Cook,Data entry & Back office,Delivery & Collection,Designer,Dogs,Driver,Drivers & Taxi,Education & Classes,Electronics & Computer,Fishes & Aquarium,For Rent,For Rent: Houses & Apartments,For Rent: Shops & Offices,For Sale,For Sale: Houses & Apartments,For Sale: Shops & Offices,Fridges,Games & Entertainment,Gym & Fitness,"Hard Disks, Printers & Monitors",Health & Beauty,Home Decor & Garden,Hotel & Travel Executive,IT Engineer & Developer,Kids,Kids Furniture,Kitchen & Other Appliances,Lands & Plots,Men,Mobile Phones,Motorcycles,Musical Instruments,Office Assistant,Operator & Technician,Other Hobbies,Other Household Items,Other Pets,Other Services,Other Vehicles,Others,PG & Guest Houses,Pet Food & Accessories,Receptionist & Front office,Sales & Marketing,Scooters,Sofa & Dining,Spare Parts,Sports Equipment,"TVs, Video - Audio",Tablets,Teacher,Washing Machines,Women
category_l2_name_en,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
ACs,2.535712,0.694861,0.077812,0.193108,2.529233,0.892994,0.069032,0.814395,18.643765,1.399592,0.312642,1.449186,0.141515,0.576659,0.397660,0.032275,0.000000,0.604756,0.025675,0.013201,0.036999,0.075875,0.000545,2.910483,0.470931,0.000363,2.335277,0.220297,1.802641,0.422548,0.430238,0.333775,0.023011,0.583865,0.118323,0.039481,0.090468,0.416371,3.373178,1.206605,1.029484,22.589969,10.671850,0.296837,0.324511,0.095615,0.237555,1.541410,0.073937,0.218783,0.523976,1.205152,0.337106,0.019014,0.150477,0.578294,4.377047,2.542676,0.670639,0.237191,4.026741,0.233074,0.072060,0.785874,0.865382
Accessories,0.531807,2.056615,0.076040,0.191655,0.949475,0.579602,0.043000,1.120684,6.837292,0.515419,0.428154,1.972302,0.100017,0.567219,0.377461,0.027507,0.000000,0.490019,0.018075,0.011066,0.031301,0.077463,0.000263,2.250589,0.342629,0.000263,1.787866,0.157245,0.605634,0.487911,0.287825,0.306427,0.014860,0.288247,0.121042,0.042420,0.057070,0.207569,1.609491,0.894618,1.036528,53.083579,7.158737,0.235972,0.322288,0.085473,0.203353,0.721934,0.065132,0.163990,0.183434,1.187871,0.263216,0.020393,0.143649,0.570802,2.219130,0.885291,0.571803,0.195818,2.815911,0.491020,0.063182,0.302106,0.545244
Accountant,0.183161,0.239827,1.533689,2.023359,0.513138,0.255853,0.016885,0.253564,6.400339,1.028278,0.056665,0.511420,0.811633,4.664314,2.853880,0.250702,0.000286,2.843864,0.051228,0.020606,0.012306,0.090436,0.001431,11.250676,1.524817,0.000286,8.273734,0.654801,0.295634,0.052945,0.096446,0.088433,0.030622,0.101883,1.322767,0.411254,0.017744,0.065824,0.460479,4.243043,0.214928,9.982857,5.823381,0.067541,3.776555,0.845117,0.038636,0.314522,0.030908,0.387214,0.258143,11.384613,1.284132,0.015740,1.803565,5.804206,1.887419,0.400951,0.220938,0.040353,0.923533,0.090722,0.532026,0.126210,0.237537
BPO & Telecaller,0.297808,0.347258,0.632888,2.884899,0.800613,0.370692,0.024356,0.441176,8.506347,1.069083,0.112739,0.851354,0.771828,4.150860,2.865156,0.187099,0.000000,2.471031,0.062920,0.020666,0.015684,0.035058,0.001107,8.391210,1.082921,0.000185,5.603366,0.463318,0.409071,0.122518,0.131560,0.111632,0.042254,0.131006,1.224260,0.347627,0.029707,0.103698,0.753746,2.814599,0.426415,13.531257,7.470662,0.120120,3.382353,0.739907,0.066610,0.367555,0.028415,0.439331,0.293380,10.852092,1.044727,0.008672,1.805668,5.494317,1.911765,0.537125,0.303712,0.076020,1.106539,0.107573,0.525685,0.220496,0.456307
Beds & Wardrobes,1.418275,0.607355,0.091530,0.206392,3.994984,1.087308,0.073954,0.738971,16.029427,1.409281,0.267857,1.391602,0.166820,0.582789,0.419105,0.036643,0.000051,0.598824,0.022510,0.014184,0.031812,0.136549,0.000617,3.353351,0.474968,0.000565,2.246257,0.225304,1.801610,0.362624,0.416895,0.289853,0.027238,0.751099,0.143642,0.044300,0.116147,0.481855,3.176047,1.143685,1.077954,22.257306,10.890244,0.334461,0.384569,0.109363,0.223094,2.015300,0.077551,0.22509

In [126]:
df1.style.background_gradient(cmap='coolwarm')

## L1 matrix

In [74]:
query = """

drop table if exists repliers;
select
  panamera_user_sk
   into temp table repliers
from ods.fact_replies
where country_sk='olx|asia|in'
 and reply_num_panamera=1
and date_sent_nk between '2019-07-01' and '2019-10-01'
order by random() limit 50000

;drop table if exists users_replies;
select
  a.panamera_user_sk,
  category_l1_name_en,
  count(distinct reply_nk) as replies
    into temp table users_replies
   from ods.fact_replies a
    left join ods.dim_categories b on a.category_sk=b.category_sk and b.country_sk='olx|asia|in'
    join repliers c on a.panamera_user_sk=c.panamera_user_sk
 where
       a.country_sk='olx|asia|in'
   and reply_num_panamera=1
   and date_sent_nk between '2019-07-01' and '2019-10-01'
group by 1,2

;drop table if exists users_replies2;
select
  a.panamera_user_sk ,
  category_l1_name_en category_l1_name_en2,
  count(distinct reply_nk) as replies2
    into temp table users_replies2
   from ods.fact_replies a
    left join ods.dim_categories b on a.category_sk=b.category_sk and b.country_sk='olx|asia|in'
    join repliers c on a.panamera_user_sk=c.panamera_user_sk
 where
       a.country_sk='olx|asia|in'
   and reply_num_panamera=1
   and date_sent_nk between '2019-07-01' and '2019-10-01'
group by 1,2

;drop table if exists df;
select
  a.panamera_user_sk,
  a.category_l1_name_en,
  b.category_l1_name_en2,
  b.replies2
 into temp table df
    from users_replies a
 join users_replies2 b on a.panamera_user_sk=b.panamera_user_sk
order by a.panamera_user_sk,a.category_l1_name_en;

select * from df



"""

In [75]:
dfl1 = pd.read_sql_query(query.replace('%', '%%'), engine)

In [88]:
dfl1.head()

,panamera_user_sk,category_l1_name_en,category_l1_name_en2,replies2
0,olx|asia|in|platform|id|100112979,Properties,Properties,1
1,olx|asia|in|platform|id|101910923,Electronics & Appliances,Used Cars,2
2,olx|asia|in|platform|id|101910923,Electronics & Appliances,Properties,80
3,olx|asia|in|platform|id|101910923,Electronics & Appliances,Electronics & Appliances,1
4,olx|asia|in|platform|id|101910923,Electronics & Appliances,Mobiles,6


In [122]:
dfl11 = dfl1.pivot_table(index='category_l1_name_en' , columns='category_l1_name_en2' , values='replies2' , aggfunc='sum').fillna(0)
dfl11 = dfl11.apply(lambda x:(x/sum(x)) * 100 , axis=1)

Usuarios que hicieron replies en X categoria tmb hicieron en Y,Z,W... categoria , en tanto %

In [123]:
dfl11.style.background_gradient(cmap='coolwarm')


category_l1_name_en2,Bikes,"Books, Sports & Hobbies",Electronics & Appliances,Fashion,Furniture,Jobs,Mobiles,Pets,Properties,Services,Used Cars
category_l1_name_en,,,,,,,,,,,
Bikes,24.1626,0.788948,8.56881,1.39135,4.05516,4.20601,27.2748,0.149739,4.21406,0.250901,24.9377
"Books, Sports & Hobbies",16.6257,2.16582,13.5926,2.8001,6.71789,4.61885,28.7223,0.234915,6.34564,0.388331,17.7879
Electronics & Appliances,17.3933,0.991817,11.5331,1.76188,5.12442,4.09465,32.6565,0.187446,4.80735,0.308712,21.1408
Fashion,16.6551,1.26124,11.9263,3.65859,6.22859,5.15276,31.4421,0.223562,6.29017,0.362349,16.7993
Furniture,17.4264,1.23875,12.5235,2.25912,8.02384,4.392,25.6285,0.239458,6.22566,0.374915,21.6678
Jobs,16.0059,0.758778,7.89493,1.88155,3.86448,17.6736,23.7445,0.150854,9.04574,0.541843,18.4379
Mobiles,16.1458,0.809752,9.19729,1.55397,3.62947,4.06197,43.323,0.160457,3.8649,0.261902,16.9915
Pets,11.3239,1.70744,10.3626,2.90342,6.9472,14.3911,16.0579,2.28407,21.9908,0.511708,11.5199
Properties,16.1629,0.901714,9.15648,2.10362,5.39459,7.28004,22.2967,0.17918,12.4934,0.434098,23.5973


# Transition matrix with view_item

In [95]:
query = """

drop table if exists session_longs;
select
 session_long
  into temp table session_longs
from ods.panameraolx_asia_hydra_ninja_android_last_3_months
 where trackevent='view_item'
group by 1
order by random() limit 50000

;drop table if exists view_item_users;
select
 c.session_long,
 category_l1_name_en,
 count(trackevent) as qty_view_item
  into temp table view_item_users
    from ods.panameraolx_asia_hydra_ninja_android_last_3_months a
     left join ods.dim_categories b on a.category_sk=b.category_sk and b.country_sk='olx|asia|in'
     join session_longs c on a.session_long=c.session_long
 where a.country_sk='olx|asia|in'
  and trackevent = 'view_item'
group by 1,2

;drop table if exists view_item_users2;
select
    session_long,
    category_l1_name_en category_l1_name_en2,
    qty_view_item qty_view_item2
into temp table view_item_users2
  from view_item_users

;drop table if exists df;
select
  a.session_long,
  a.category_l1_name_en,
  b.category_l1_name_en2,
  b.qty_view_item2
 into temp table df
    from view_item_users a
 join view_item_users2 b on a.session_long=b.session_long
order by a.session_long,a.category_l1_name_en;

select * from df



"""

In [96]:
dfvi = pd.read_sql_query(query.replace('%', '%%'), engine)

In [101]:
dfvi.head()

,session_long,category_l1_name_en,category_l1_name_en2,qty_view_item2
0,0749ea2b96461b11,Bikes,Properties,29
1,0749ea2b96461b11,Bikes,Bikes,37
2,0749ea2b96461b11,Bikes,Jobs,3
3,0749ea2b96461b11,Jobs,Properties,29
4,0749ea2b96461b11,Jobs,Bikes,37


In [118]:
dfvi1 = dfvi.pivot_table(index='category_l1_name_en' , columns='category_l1_name_en2' , values='qty_view_item2' , aggfunc='sum').fillna(0)
dfvi1 = dfvi1.apply(lambda x:x/sum(x) , axis=1)

In [119]:
dfvi1.head()

category_l1_name_en2,Bikes,"Books, Sports & Hobbies",Electronics & Appliances,Fashion,Furniture,Jobs,Mobiles,Pets,Properties,Services,Used Cars
category_l1_name_en,,,,,,,,,,,
Bikes,0.229267,0.012036,0.089892,0.017973,0.052247,0.032904,0.139831,0.004027,0.086698,0.003692,0.331433
"Books, Sports & Hobbies",0.183266,0.021367,0.113957,0.024008,0.065609,0.032621,0.148313,0.004244,0.096318,0.004804,0.305493
Electronics & Appliances,0.181007,0.013149,0.110762,0.020046,0.061951,0.036392,0.148684,0.004178,0.098948,0.004134,0.320749
Fashion,0.180964,0.014934,0.105749,0.029630,0.066488,0.036348,0.153883,0.004010,0.104884,0.004380,0.298730
Furniture,0.172982,0.013726,0.105902,0.021868,0.082283,0.036317,0.129049,0.004856,0.113590,0.004484,0.314943


In [117]:
#dfvi1.to_csv('transition_matrix_view_item_l1.tsv')

In [ ]:
De todos los view items de todas las personas que vieron al menos Bikes , el 15% de esos view items corresponden a Bikes 

In [121]:
dfvi1.style.background_gradient(cmap='coolwarm')

category_l1_name_en2,Bikes,"Books, Sports & Hobbies",Electronics & Appliances,Fashion,Furniture,Jobs,Mobiles,Pets,Properties,Services,Used Cars
category_l1_name_en,,,,,,,,,,,
Bikes,0.229267,0.0120362,0.0898923,0.0179727,0.0522474,0.0329038,0.139831,0.00402713,0.0866978,0.00369175,0.331433
"Books, Sports & Hobbies",0.183266,0.0213666,0.113957,0.0240083,0.0656088,0.0326214,0.148313,0.00424378,0.0963182,0.0048045,0.305493
Electronics & Appliances,0.181007,0.0131486,0.110762,0.020046,0.0619514,0.036392,0.148684,0.00417763,0.0989483,0.00413392,0.320749
Fashion,0.180964,0.014934,0.105749,0.0296298,0.0664879,0.0363481,0.153883,0.00401026,0.104884,0.00437953,0.29873
Furniture,0.172982,0.0137262,0.105902,0.0218677,0.082283,0.0363173,0.129049,0.00485586,0.11359,0.00448427,0.314943
Jobs,0.172397,0.012815,0.0929705,0.0206665,0.0548606,0.0859692,0.12939,0.00301822,0.118857,0.00574157,0.303315
Mobiles,0.18695,0.0128516,0.0977888,0.0197767,0.0527973,0.0354409,0.193961,0.00416817,0.0896825,0.00379822,0.302785
Pets,0.144878,0.0195463,0.14471,0.0300372,0.0753793,0.029726,0.122086,0.0478132,0.105456,0.0104988,0.269869
Properties,0.172447,0.0126458,0.0938887,0.0198164,0.0597519,0.0404882,0.131194,0.00369637,0.136811,0.00424691,0.325013
